Lx599
0.1 -> GRN, Rcistarget, around 1000 cells x 200 genes
0.25 -> GRN, Rcistarget, around 2000 cells
0.5


Lx108
0.2 -> GRN, Rcistarget, around 1000 cells x 2000 genes
0,4 -> GRN

In [1]:
#STEP 1: importing all needed moduels

import os, glob, re, pickle
from functools import partial
from collections import OrderedDict
import operator as op
from cytoolz import compose

import pandas as pd
import seaborn as sns
import numpy as np
import scanpy as sc
import anndata as ad
import matplotlib as mpl
import matplotlib.pyplot as plt
import skmisc

from pyscenic.export import export2loom, add_scenic_metadata
from pyscenic.utils import load_motifs
from pyscenic.transform import df2regulons
from pyscenic.aucell import aucell
from pyscenic.binarization import binarize
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_binarization, plot_rss

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
from pyscenic.utils import modules_from_adjacencies
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell
from dask.diagnostics import ProgressBar
from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase

#STEP 1+: Set maximum number of jobs
sc.settings.njobs = 32

/home/linl5/miniconda3/envs/pyscenic/lib/python3.10/site-packages/loompy/bus_file.py:68: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def twobit_to_dna(twobit: int, size: int) -> str:
/home/linl5/miniconda3/envs/pyscenic/lib/python3.10/site-packages/loompy/bus_file.py:85: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def dna_to_twobit(dna: str) -> int:
/home/li

Divide each file into by states and then running each state for network reconstruction
- A is patient pretreatment pdx
- B is patient posttreatment pdx

In [38]:
#prepping
DATASET_ID = 'lx108'
GENE_NUM = 2000

In [39]:
#STEP 1++: preparing pathway constant variables for easy coding

SOHRAB_RESOURCES_FOLDERNAME = "/work/shah/users/salehis/projects/cdm/data/sclc/{}".format(DATASET_ID)
RESULTS_FOLDERNAME = "/home/linl5/project/SCLC/results/{}".format(DATASET_ID)
FIGURES_FOLDERNAME = "/home/linl5/project/SCLC/figures"
AUXILLIARIES_FOLDERNAME = "/home/linl5/project/SCLC/auxilliaries"
RESOURCES_FOLDERNAME = "/home/linl5/project/SCLC/resources"
DATA_FOLDERNAME = "/home/linl5/project/SCLC/data/{}".format(DATASET_ID)

In [40]:
#Downloaded fromm pySCENIC github repo: https://github.com/aertslab/pySCENIC/tree/master/resources Aug-1-2023
#lambert2018.txt used in their cancer patient tutorial
HUMAN_TFS_FNAME = os.path.join(RESOURCES_FOLDERNAME, 'allTFs_hg38.txt')

In [41]:
#STEP 2: Downloading metadata and expression matrix
#input: expression matrix and metadata file

#organized by cell ID and Gene matrix, values are counts of read in that cell
COUNTS_MTX_FNAME = os.path.join(SOHRAB_RESOURCES_FOLDERNAME, 'rna.h5ad')
COUNTS_MTX_FNAME

'/work/shah/users/salehis/projects/cdm/data/sclc/lx108/rna.h5ad'

In [42]:
#STEP 3: Importing and Analyzing the rna DATA

# Read the H5AD file using anndata
adata = ad.read(COUNTS_MTX_FNAME)
COUNTS_MTX_FNAME

'/work/shah/users/salehis/projects/cdm/data/sclc/lx108/rna.h5ad'

In [43]:
#STEP4: Understanding Dataset

#print out information about anndata
print(adata)

#print out shape
print("Shape of expression matrix: ", adata.shape)

# the columns (variables) of the expression matrix
print("Columns (variables):")
print(adata.var)

# the rows (observations) of the expression matrix
print("\nRows (observations):")
print(adata.obs)


AnnData object with n_obs × n_vars = 18481 × 33538
    obs: 'timepoint', 'datatag', 'batch', 'state'
    var: 'gene_ids', 'feature_types'
Shape of expression matrix:  (18481, 33538)
Columns (variables):
                    gene_ids    feature_types
MIR1302-2HG  ENSG00000243485  Gene Expression
FAM138A      ENSG00000237613  Gene Expression
OR4F5        ENSG00000186092  Gene Expression
AL627309.1   ENSG00000238009  Gene Expression
AL627309.3   ENSG00000239945  Gene Expression
...                      ...              ...
AC233755.2   ENSG00000277856  Gene Expression
AC233755.1   ENSG00000275063  Gene Expression
AC240274.1   ENSG00000271254  Gene Expression
AC213203.1   ENSG00000277475  Gene Expression
FAM231C      ENSG00000268674  Gene Expression

[33538 rows x 2 columns]

Rows (observations):
                              timepoint datatag batch state
AAACCCAAGTCGCTAT-1_Lx108_UUUU      UUUU   Lx108     0  UUUU
AAACCCAGTTGTGCCG-1_Lx108_UUUU      UUUU   Lx108     0  UUUU
AAACCCATCTCGTGAA-

In [44]:
#STEP 5: preprocessing and filtering

#make gene name unique
adata.var_names_make_unique()

#processing out data-prefilter
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

#saving a copy of the power filtered count into raw
adata.raw = adata

#post filer: previous was n_obs × n_vars = 33207 × 33538
print("Post Filter: ", adata.shape)


Post Filter:  (18436, 22723)


LX599: 
- Pre-Filter: 33207 × 33538
- Post Filter:  (33108, 28701)


Lx108: 
- Pre-Filter: 18481 × 33538
- Post Filter: (18436, 22723)

Lx33:
- pre filter: 23691 × 33538
- Post Filter:  (23051, 22899)

In [45]:
# STEP 6: Prepping for timepoint splitting
adata.layers['counts'] = adata.X
adata.raw = adata


In [46]:
#STEP 7: subsampling time point (IDEA-> in original sample filte by hvg, then split the following into states, then cluster and subsample from cluster)

# set the n_top_genes as necessary
sc.pp.highly_variable_genes(adata, n_top_genes=GENE_NUM, subset=True, flavor='seurat_v3')

#Splitting the cells by states
unique_state = adata.obs['state'].unique()
print(unique_state)

#storing
adata_by_state = {}

#selecting out by TP
for state in unique_state:
    adata_subset = adata[adata.obs['state'] == state].copy()
    adata_by_state[state] = adata_subset
    print(state, ": ", adata_by_state[state].shape)

    

['UUUU', 'UUUT', 'UU', 'UTTU']
Categories (4, object): ['UTTU', 'UU', 'UUUT', 'UUUU']
UUUU :  (4128, 2000)
UUUT :  (4564, 2000)
UU :  (5035, 2000)
UTTU :  (4709, 2000)


lx33 (post cluster):
- UTTU :  (4021, 2000)
- UU :  (6176, 2000)
- UUUT :  (6619, 2000)
- UUUU :  (6235, 2000)


The Leiden algorithm improves upon Louvain by using the "agglomerative" approach to optimize a different quality function known as the "improved modularity." Improved modularity has a resolution parameter that allows Leiden to control the granularity of the clustering solution. It also uses a smart local move algorithm to avoid some of the resolution limit issues present in Louvain. Higher paramter means more identified communities. 

#The Leiden algorithm is a hierarchical clustering algorithm, that recursively merges communities into single nodes by greedily optimizing the modularity and the process repeats in the condensed graph.
#The Leiden algorithm improves upon Louvain by using the "agglomerative" approach to optimize a different quality function known as the "improved modularity." Improved modularity has a resolution parameter that allows Leiden to control the granularity of the clustering solution. It also uses a smart local move algorithm to avoid some of the resolution limit issues present in Louvain.

In [47]:
#STEP 8: cluster definition

def cluster_rna(bdata):
    #I have already did filtering and HVG selection on main anndata, we want to keep same hvg across our states
    sc.pp.normalize_total(bdata)    
    sc.pp.log1p(bdata)
    sc.pp.pca(bdata)
    sc.pp.neighbors(bdata)
    sc.tl.umap(bdata)
    sc.tl.leiden(bdata, resolution=2)
    return bdata

In [48]:
#STEP 9: Clustering

for state in unique_state:
    adata_by_state[state] = cluster_rna(adata_by_state[state])

In [49]:
#STEP 10: Subsampling

from tqdm import tqdm
frac_cells = 0.4
data_sub = {}

#Consistent randomization
np.random.seed(0)

for state in unique_state:
    sub_cells = []
    for clust in tqdm(adata_by_state[state].obs['leiden'].unique()):
        # sample 10% of cells from each cluster
        cells_in_clust = adata_by_state[state].obs_names[adata_by_state[state].obs['leiden'] == clust].copy()
        #dropping out clusters with less than 5 cells
        if (len(cells_in_clust) > 5):
            chosen_cells = np.random.choice(cells_in_clust, size=int(len(cells_in_clust)*frac_cells), replace=False)
            sub_cells.extend(chosen_cells)
    data_sub[state] = adata_by_state[state][sub_cells, :].copy()

100%|██████████| 20/20 [00:00<00:00, 3987.55it/s]


In [52]:
#checking for subsampling population

for state in unique_state:
    print(state, data_sub[state].shape)
    set1 = set(data_sub[state].var_names)
    set2 = set(data_sub[unique_state[1]].var_names)
    print("Same Gene set check: ", state, unique_state[1], len(set1.intersection(set2)))

print(COUNTS_MTX_FNAME)

UUUU (1643, 2000)
Same Gene set check:  UUUU UUUT 2000
UUUT (1818, 2000)
Same Gene set check:  UUUT UUUT 2000
UU (2004, 2000)
Same Gene set check:  UU UUUT 2000
UTTU (1878, 2000)
Same Gene set check:  UTTU UUUT 2000
/work/shah/users/salehis/projects/cdm/data/sclc/lx108/rna.h5ad


LX599

10%
UUa (715, 2000)
Same Gene set check:  UUa UUa  2000
UTb (515, 2000)
Same Gene set check:  UTb UUa  2000
UUb (991, 2000)
Same Gene set check:  UUb UUa  2000
UTa (1042, 2000)
Same Gene set check:  UTa UUa  2000

25%
UUa (1812, 2000)
Same Gene set check:  UUa UUa  2000
UTb (1307, 2000)
Same Gene set check:  UTb UUa  2000
UUb (2500, 2000)
Same Gene set check:  UUb UUa  2000
UTa (2618, 2000)
Same Gene set check:  UTa UUa  2000

50%
UUa (3633, 2000)
Same Gene set check:  UUa UUa  2000
UTb (2633, 2000)
Same Gene set check:  UTb UUa  2000
UUb (5011, 2000)
Same Gene set check:  UUb UUa  2000
UTa (5248, 2000)
Same Gene set check:  UTa UUa  2000

LX108
20%
UUUU (818, 2000)
Same Gene set check:  UUUU UUUT 2000
UUUT (904, 2000)
Same Gene set check:  UUUT UUUT 2000
UU (998, 2000)
Same Gene set check:  UU UUUT 2000
UTTU (934, 2000)
Same Gene set check:  UTTU UUUT 2000

40%
UUUU (1643, 2000)
Same Gene set check:  UUUU UUUT 2000
UUUT (1818, 2000)
Same Gene set check:  UUUT UUUT 2000
UU (2004, 2000)
Same Gene set check:  UU UUUT 2000
UTTU (1878, 2000)
Same Gene set check:  UTTU UUUT 2000

LX33

15%
UTTU (594, 2000)
Same Gene set check:  UTTU UU 2000
UU (917, 2000)
Same Gene set check:  UU UU 2000
UUUT (981, 2000)
Same Gene set check:  UUUT UU 2000
UUUU (922, 2000)
Same Gene set check:  UUUU UU 2000

In [53]:
#STEP 11: copying over raw count 

for state in unique_state:
    data_sub[state].X = data_sub[state].layers['counts'].copy()
    
#check for sample output
data_sub[unique_state[1]].to_df()

,FAM87B,LINC00115,HES4,ISG15,ATAD3C,GABRD,AL590822.2,HES5,AL139246.5,AL139246.3,...,BACE2,TFF1,AP001626.1,ERVH48-1,FRGCA,AIRE,TRPM2,C21orf58,PCNT,MT-ND6
GGAGGTAAGTCGCCCA-1_Lx108_UUUT,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,1.0
ATTGTTCAGCATTTCG-1_Lx108_UUUT,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,4.0
GAGGCCTCATCGCTAA-1_Lx108_UUUT,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.0
ATCCATTGTTGAATCC-1_Lx108_UUUT,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0
GGAGAACAGCCTGTCG-1_Lx108_UUUT,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCACGCTAGGGTTGCA-1_Lx108_UUUT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0
AGTTCCCTCGGTGTAT-1_Lx108_UUUT,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0
ATGTCCCAGAGCATTA-1_Lx108_UUUT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CTAACTTTCTTGCAAG-1_Lx108_UUUT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [54]:
#STEP 12: output Timepoint specific count matrix after subsampling by state
for state in unique_state:
    EXP_MTX_QC_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.{}.{}.qc.tpm.csv'.format(DATASET_ID, state, frac_cells))
    data_sub[state].to_df().to_csv(EXP_MTX_QC_FNAME, index=False, sep='\t')
    print("Finish with: ", EXP_MTX_QC_FNAME, " Shape: ", data_sub[state].shape)


Finish with:  /home/linl5/project/SCLC/results/lx108/lx108.UUUU.0.4.qc.tpm.csv  Shape:  (1643, 2000)
Finish with:  /home/linl5/project/SCLC/results/lx108/lx108.UUUT.0.4.qc.tpm.csv  Shape:  (1818, 2000)
Finish with:  /home/linl5/project/SCLC/results/lx108/lx108.UU.0.4.qc.tpm.csv  Shape:  (2004, 2000)
Finish with:  /home/linl5/project/SCLC/results/lx108/lx108.UTTU.0.4.qc.tpm.csv  Shape:  (1878, 2000)


In [55]:
# STEP 13: prepping for GRN, Loading in expression matrix and TF files

#Loading TF
tf_names = load_tf_names(HUMAN_TFS_FNAME)
print(HUMAN_TFS_FNAME, ": Size of TF list", len(tf_names))

#selecting !!!! CHANGE
frac_cells = 0.4

#expression matrix
for state in unique_state:
    EXP_MTX_QC_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.{}.{}.qc.tpm.csv'.format(DATASET_ID, state, frac_cells))
    ex_matrix = pd.read_csv(EXP_MTX_QC_FNAME, sep='\t', header=0, index_col=0)
   
    #Input Checking 
    print("\nExpression matrix shape for", DATASET_ID, state, ex_matrix.shape)

#STEP 14: Running GRNBOOST2 for coexpression modules
    adjacencies = grnboost2(expression_data=ex_matrix, tf_names=tf_names, verbose=True)
    print("\nCOMPLETED GRNBOOST2 RUNNING FOR", DATASET_ID, state)
    print("\n", adjacencies.head())
    ADJACENCIES_FNAME = os.path.join(DATA_FOLDERNAME, "{}.{}.{}.adjacencies.tsv".format(DATASET_ID, state, frac_cells))
    adjacencies.to_csv(ADJACENCIES_FNAME, index=False, sep='\t')
    print("SUCCESSFUL WRITING TO", ADJACENCIES_FNAME, "\n")
    

/home/linl5/project/SCLC/resources/allTFs_hg38.txt : Size of TF list 1892

Expression matrix shape for lx108 UUUU (1643, 1999)
preparing dask client
parsing input
creating dask graph
8 partitions
computing dask graph
shutting down client and local cluster
finished

COMPLETED GRNBOOST2 RUNNING FOR lx108 UUUU

        TF  target  importance
2    ENO1  IGFBP2  258.258363
2    ENO1    LDHA  240.651561
2    ENO1   BNIP3  205.784701
98  RPL35    RPL8  138.692116
2    ENO1   NDRG1  128.348464
SUCCESSFUL WRITING TO /home/linl5/project/SCLC/data/lx108/lx108.UUUU.0.4.adjacencies.tsv 


Expression matrix shape for lx108 UUUT (1818, 1999)
preparing dask client
parsing input
creating dask graph
8 partitions
computing dask graph
shutting down client and local cluster
finished

COMPLETED GRNBOOST2 RUNNING FOR lx108 UUUT

         TF  target  importance
2     ENO1  IGFBP2  236.044854
2     ENO1    LDHA  182.164460
2     ENO1   BNIP3  145.333614
98   RPL35   RPL36  120.979624
165   E2F1    PCNA  120.93

In [56]:
#STEP 15: Prepping for RCistarget: Loading Database and motif

DATABASE_FOLDER = "/home/linl5/project/SCLC/auxilliaries/"
DATABASES_GLOB = os.path.join(DATABASE_FOLDER, "hg38_*.mc9nr.genes_vs_motifs.rankings.feather")

db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [RankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs

MOTIF_ANNOTATIONS_FNAME = os.path.join(RESOURCES_FOLDERNAME,"motifs-v9-nr.hgnc-m0.001-o0.0.tbl")

In [ ]:
#STEP 16: Running RCistarget

for state in unique_state:
    #reading necessary files for rcistarget
    ADJACENCIES_FNAME = os.path.join(DATA_FOLDERNAME, "{}.{}.{}.adjacencies.tsv".format(DATASET_ID, state, frac_cells))
    adjacencies = pd.read_csv(ADJACENCIES_FNAME, sep='\t')
    print("\nFINISHED READING ADJACENCIES FILE", ADJACENCIES_FNAME,"\n")
    EXP_MTX_QC_FNAME = os.path.join(RESULTS_FOLDERNAME, '{}.{}.{}.qc.tpm.csv'.format(DATASET_ID, state, frac_cells))
    ex_matrix = pd.read_csv(EXP_MTX_QC_FNAME, sep='\t', header=0, index_col=0)
    print("\nFINISHED READING EXPRESSION MATRIX", EXP_MTX_QC_FNAME,"\n")
    MODULES_FNAME = os.path.join(DATA_FOLDERNAME, '{}.{}.{}.modules.p'.format(DATASET_ID, state, frac_cells))
    MOTIFS_FNAME = os.path.join(DATA_FOLDERNAME, '{}.{}.{}.motifs.csv'.format(DATASET_ID, state, frac_cells))
    REGULONS_FNAME = os.path.join(DATA_FOLDERNAME, '{}.{}.{}.regulons.p'.format(DATASET_ID, state, frac_cells))

    #making modules from adjacencies
    modules = list(modules_from_adjacencies(adjacencies, ex_matrix))
    
    #writing modules object to file
    with open(MODULES_FNAME, 'wb') as f:
        pickle.dump(modules, f)
    print("\nCOMPLETED COEXPRESSION MODULE WRITING:", MODULES_FNAME,"\n")
    
    #running Rcistarget with progress bar: searching for enriched motifs and true candidate genes
    with ProgressBar():
        df = prune2df(dbs, modules, MOTIF_ANNOTATIONS_FNAME, client_or_address="dask_multiprocessing") 
    
    #writing enriched motifs with candidate target to file
    df.to_csv(MOTIFS_FNAME)
    print("\nCOMPLETED WRITING ENRICHED MOTIFS", MOTIFS_FNAME,"\n")
    print(df.head())
    
    #making regulon objects
    regulons = df2regulons(df)
    
    #writing regulon objects to file
    with open(REGULONS_FNAME, 'wb') as f:
        pickle.dump(regulons, f)
    print("\nCOMPLETED WRITING DISCOVERED REGULON", REGULONS_FNAME,"\n")


FINISHED READING ADJACENCIES FILE /home/linl5/project/SCLC/data/lx108/lx108.UUUU.0.4.adjacencies.tsv 




2023-08-07 15:44:33,310 - pyscenic.utils - INFO - Calculating Pearson correlations.

2023-08-07 15:44:33,341 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [False].



FINISHED READING EXPRESSION MATRIX /home/linl5/project/SCLC/results/lx108/lx108.UUUU.0.4.qc.tpm.csv 




2023-08-07 15:44:35,121 - pyscenic.utils - INFO - Creating modules.



COMPLETED COEXPRESSION MODULE WRITING: /home/linl5/project/SCLC/data/lx108/lx108.UUUU.0.4.modules.p 

[                                        ] | 0% Completed | 14.02 sms

...

[                                        ] | 0% Completed | 208.47 s


2023-08-07 15:48:13,002 - pyscenic.transform - WARNING - Less than 80% of the genes in ZSCAN31 could be mapped to hg38__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr.genes_vs_motifs.rankings. Skipping this module.


[                                        ] | 0% Completed | 216.74 s


2023-08-07 15:48:21,321 - pyscenic.transform - WARNING - Less than 80% of the genes in BHLHE22 could be mapped to hg38__refseq-r80__500bp_up_and_100bp_down_tss.mc9nr.genes_vs_motifs.rankings. Skipping this module.


[                                        ] | 0% Completed | 218.89 s

After grouping the data by timepoints
- How is clustering in this case difference by just making anndata subset selected by timepoint (pin)
- Do i do PCA after clustering -> pca changes the expression matrix so output to GENIE3 is not pca, just raw unnormalize, unpertrude data except basic filtering
- If you have more than one condition, it’s often helpful to perform integration to align the cells -? and then within each timepoint there would be three batches so i should find a way to remove this right